In [ ]:
!pip install transformers tensorflow

In [79]:
from transformers import DistilBertTokenizer,TFDistilBertForSequenceClassification
import tensorflow as tf
import csv


In [56]:
text = []
labels = []

with open('/content/drive/MyDrive/BTP/dataset.csv') as fp:
  reader = csv.reader(fp,delimiter=',')
  for row in reader:
    text.append(row[2])
    labels.append(row[3])
text = text[1:]
labels = labels[1:]
for i,val in enumerate(labels):
  labels[i] = int(labels[i])
tot_size = len(text)
train_ratio = 0.9
train_len=int(train_ratio*tot_size)
train_text = text[:train_len]
train_labels = labels[:train_len]

val_text = text[train_len:]
val_labels = labels[train_len:]




In [62]:
import json
training_size = 20000

with open("/content/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

train_text = sentences[0:training_size]
val_text = sentences[training_size:]
train_labels = labels[0:training_size]
val_labels = labels[training_size:]

In [80]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [81]:
train_encodings = tokenizer(train_text,truncation=True,padding=True)

val_encodings = tokenizer(val_text,truncation=True,padding=True)

In [82]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [83]:
from transformers import TFAutoModelForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_44']
You should probably TRAIN this model on a down-stream task to be able to use i

In [85]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_func, metrics=['accuracy'])
model.fit(train_dataset.shuffle(len(train_text)).batch(16),
          epochs=3,
          batch_size=16)

Epoch 1/3
1250/1250 [==============================] - 323s 249ms/step - loss: 0.2627 - accuracy: 0.8889
Epoch 2/3
1250/1250 [==============================] - 312s 249ms/step - loss: 0.0988 - accuracy: 0.9645
Epoch 3/3
1250/1250 [==============================] - 312s 250ms/step - loss: 0.0396 - accuracy: 0.9873


In [86]:
model.evaluate(val_dataset.shuffle(len(val_text)).batch(16),
               return_dict=True,
               batch_size=16)

420/420 [==============================] - 37s 80ms/step - loss: 0.2847 - accuracy: 0.9115


{'accuracy': 0.9114621877670288, 'loss': 0.2847088575363159}

In [87]:
model.save_pretrained('/content/drive/MyDrive/BTP/DistilBERT')